In [237]:
from lxml import etree
import re
from operator import itemgetter


In [238]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open

In [239]:
def extracttext(text):
    
    sentenses = re.split(r"[\n|\r]", text)
    # Remove leading and trailing spaces from each sentence
    results = []
    for sen in sentenses:
        s = sen.strip()
        if len(s):
            results.append(s)
            num_libros=len(results)
    #print("\n")
    return results[1]


In [240]:
def querysearch():
    
    queryid = []
    querytext = []
    a = 1
    queryall = [queryid ,querytext]
    while a!=47:
        if a < 10:
            b = "0"+str(a)
            if a == 4:
                a+=1
        elif a < 50:
                b = str(a)
                if a == 10 or a == 14 or a == 32 or a == 34 or a ==38 or a == 42:
                    a+=1
                elif a == 19 or a == 29:
                    a+=2

        docquer = etree.parse("./queries-raw-texts/wes2015.q"+b+".naf")
        raizquer=docquer.getroot()
        idquer = raizquer[0][1].get('publicId')
        textquery = raizquer[1].text
        queryid.append(idquer)
        querytext.append(textquery)
        a+=1
    #print(queryid)
    #print(querytext)    
    return queryall
    #print("\n")

In [241]:
#Gensim trae sus propias funciones para el procesamiento de texto
p = PorterStemmer()

<h2>Corpus Streaming</h2>
Normalmente los corpus reciden completamente en la memoria. Supongamos que hay millones de documentos en el corpus. Almacenarlos todos en RAM no será suficiente. En su lugar, supongamos que los documentos se almacenan en un archivo en el disco, un documento por línea. Gensim puede procesar un documento a la vez y actualizar dinamicamente e calculo de IDF y del vocabulario.

In [242]:
#Procesamiento de un texto utilizando las funciones de Gensim
def process(text):
    doc_nor = text.lower()
    #doc_reg = re.sub(r'[^\w|^\d]', ' ', doc_nor)
    #doc_reg = re.sub(r'[^a-zA-Z]', ' ', doc_nor)
    doc_sw = remove_stopwords(doc_nor)
    doc_stem = PorterStemmer().stem_sentence(doc_sw)
    return doc_stem.split()

In [243]:
#Creaccion de lista con documentos procesados
docDict = []
docTitle = []
docID = []
a = 1

while a!=332:
    if a < 10:
        b = "00"+str(a)
    elif a < 100:
            b = "0"+str(a)
    else:
            b = str(a)
    doc = etree.parse("./docs-raw-texts/wes2015.d"+b+".naf")
    raiz=doc.getroot()
    titulo = raiz[0][0].get('title')
    docTitle.append(titulo)
    iddoc = raiz[0][1].get('publicId')
    docID.append(iddoc)
    #print(iddoc)
    #print(titulo)
    text = raiz[1].text
    a+=1
    line = extracttext(text)
    #print("\n")
    docDict.append(process(line))

#print(docID[2])
#print(docTitle[2])
#print(docDict[2]) #Documentos ya procesados, de esta forma deben ser enviados al metodo de Gensim



In [244]:
#Ahora si llamamos al metodo de Gensim para crear el diccionario a partir de los documentos
dictionary = corpora.Dictionary(docDict)
dictionary.save('clase3.dict')
#print(dictionary)

#Notese que Gensim de una vez asigna a cada token un id
#print(dictionary.token2id)

In [245]:
new_doc = "Human media interaction media"
new_doc_bow = dictionary.doc2bow(process(new_doc))
print(new_doc_bow)  # ¿Si hay cuatro palabras pq solo dos tokens?

[(70, 1), (4236, 1), (7809, 2)]


In [246]:
#Construccion del corpus == Cada documento representado con el vocabulario/diccionario definido anteriormente (Bolsa de palabras)
class MyCorpus(object):
    def __iter__(self):
        a = 1

        while a!=332:
            if a < 10:
                b = "00"+str(a)
            elif a < 100:
                    b = "0"+str(a)
            else:
                b = str(a)
            doc = etree.parse("./docs-raw-texts/wes2015.d"+b+".naf")
            raiz=doc.getroot()
            titulo = raiz[0][0].get('title')
            iddoc = raiz[0][1].get('publicId')
            #print(iddoc)
            #print(titulo)
            text = raiz[1].text
            a+=1
            line = extracttext(text)
            yield dictionary.doc2bow(process(line))

corpus_memory_friendly = MyCorpus()#Todos mis documentos ahora estan representados como una bolsa de palabras
#Almaceno mi corpus
corpora.MmCorpus.serialize('corpus.mm', corpus_memory_friendly)  

In [247]:
#Cargo mi corpus
corpus = corpora.MmCorpus('corpus.mm')
#print(corpus) #No lo carga en memoria
#Para leer la representacion de bolda de palabras resultante de cada documento en el corpus
#for doc in corpus:
#    print(doc)

<h2>Vector Space Model - TF-IDF</h2>

In [248]:
#Suponga que tenemos el diccionario y corpus en disco
#Primero necesitamos cargarlos
dictionary = corpora.Dictionary.load('clase3.dict')
corpus = corpora.MmCorpus('corpus.mm')

In [249]:
#Construyendo un modelo VSM con ponderacion por TF-IDF
tfidf = models.TfidfModel(corpus) #Inizializacion

In [250]:
#Supongamos que queremos calcular la similitud de un nuevo documento con los documentos en el corpus
#1. Construimos la representación vectorial del query
query_doc = "Amazon vs Microsoft"
query_doc_bow = dictionary.doc2bow(process(query_doc))
print(query_doc_bow)
print(tfidf[query_doc_bow])

[(16449, 1)]
[(16449, 1.0)]


In [251]:
#2. Ahora construimos la matrix t/d con los documentos contra los cuales queremos compara el query
index = similarities.MatrixSimilarity(tfidf[corpus]) 
# Vamos a salvar el inidice resultante para no tener que recalcularlo cada vez que los necesitemos.
index.save('clase3tfidf.index')

In [252]:
#Una vez guardado lo podemos cargar sin necesidad de recalcularlo
index = similarities.MatrixSimilarity.load('clase3tfidf.index')

In [253]:
# Ejecutar las similitudes entre el documento query y el corpus ahora es muy facil!!!

queryreturn = querysearch()
queryid = queryreturn[0]
querydoc = queryreturn[1]
archivo = open ('top-10.txt','w')
id=0
for query in querydoc:
    
    query_doc_bow = dictionary.doc2bow(process(query))
    sims = index[tfidf[query_doc_bow]]
    #print(type(sims))
    listofdoctf= list(enumerate(sims))
    #print(listofdoctf)
    #print("\n")
    sorted_by_second = sorted(listofdoctf,key=itemgetter(1), reverse = True)
    #print(sorted_by_second)
    varTen = "10 most similarity documents:\n"
    #archivo.write(varTen)
    archivo.write(queryid[id]+"\t")
    for doc, similitudQ in sorted_by_second[:10]:  
        
        archivo.write( docID[doc]+",")
        #print("{}\t{}".format(docID[doc], similitudQ))
    #print("\n")    
    archivo.write("\n")
    id+=1
archivo.close()
    

#print(list(enumerate(sims)))  print (document_number, document_similarity) 2-tuples#

In [254]:
dicFinal = []
dicFinal2 = []

def relevancejud(docDic):
    
    #print(len(docDic))
    
    for index in range(len(docDic)):
        dataqueryid = docDic[index][0]
        dataquery = docDic[index][1]
        #print(dataqueryid)
        #print("\n")
        #print(dataquery)
        #print("\n")
        normalice = re.sub(r':\d', '', dataquery)
        normalice = re.split(",",normalice)
        #print(normalice)
        dicFinal.append(dataqueryid)
        dicFinal2.append(normalice)

        #print(dicFinal)
        #print(dicFinal2)


In [255]:
i=0
for a in dicFinal2:
    print (dicFinal[i])
    for b in a:
        print (b)
    i+=1

In [256]:
dicFinal3 = []
dicFinal4 = []
def docssimulationDict(docDic):
    
    #print(len(docDic))
    
    for index in range(len(docDic)):
        dataqueryid = docDic[index][0]
        dataquery = docDic[index][1]
        #print(dataqueryid)
        #print("\n")
        #print(dataquery)
        #print("\n")
        normalice = re.sub(r':\d', '', dataquery)
        normalice = re.split(",",normalice)
        #print(normalice)
        dicFinal3.append(dataqueryid)
        dicFinal4.append(normalice)

        #print(dicFinal3)
        #print(dicFinal4)


In [257]:
docDict1 = []
for line in smart_open('top-10.txt', 'rb'):
    docDict1.append(process(line))  

docDict2 = []
for line in smart_open('relevance-judgments.tsv', 'rb'):
    docDict2.append(process(line))  
#print(docDict2)
    

relevancejud(docDict2)
docssimulationDict(docDict1)
print(dicFinal)
print(dicFinal3)

['q01', 'q02', 'q03', 'q04', 'q06', 'q07', 'q08', 'q09', 'q10', 'q12', 'q13', 'q14', 'q16', 'q17', 'q18', 'q19', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q32', 'q34', 'q36', 'q37', 'q38', 'q40', 'q41', 'q42', 'q44', 'q45', 'q46']
['q01', 'q02', 'q03', 'q04', 'q06', 'q07', 'q08', 'q09', 'q10', 'q12', 'q13', 'q14', 'q16', 'q17', 'q18', 'q19', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q32', 'q34', 'q36', 'q37', 'q38', 'q40', 'q41', 'q42', 'q44', 'q45', 'q46']


/home/felipe/Documentos/venv/lib/python3.6/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
